# Sequential Importance Sampling (SIS)

heres an algorithm from a big book, Probabilistic Machine Learning Advanced Topics:

![image](images/Screenshot%20from%202024-10-17%2013-54-06.png)

Actually implement simons alg for static tracking first as it will be easier:

![images](images/GreenMaskAlg1.png)
![images](images/GreenMaskAlg2.png)